4차원 데이터 예제

In [2]:
import numpy as np

x = np.random.rand(10, 1, 28, 28) #높이 28 너비 28 채널 1 데이터 10
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape

(1, 28, 28)

In [4]:
x[0, 0]

array([[3.95619900e-01, 6.84264127e-01, 8.87198341e-01, 2.60373917e-01,
        3.39625392e-01, 6.32548171e-01, 6.53271128e-01, 2.04084097e-01,
        7.29174832e-01, 1.60594785e-01, 5.57770820e-01, 8.71882267e-01,
        5.27569602e-01, 1.99991744e-01, 7.37171593e-01, 1.78635462e-01,
        7.69145098e-01, 4.70487136e-01, 2.43015874e-01, 4.32862960e-01,
        8.05153997e-02, 6.68980014e-01, 5.55812428e-01, 7.16438258e-01,
        9.27367115e-01, 8.16486130e-02, 1.35170819e-02, 3.29020028e-01],
       [2.59080951e-01, 4.32251657e-01, 2.81063579e-02, 8.41660956e-01,
        8.97381448e-01, 5.14286477e-01, 4.20626943e-01, 3.21352246e-01,
        8.24584873e-01, 5.47405927e-01, 4.82773458e-04, 3.59723505e-01,
        9.01920111e-01, 9.95464104e-02, 4.63101196e-01, 8.34734159e-01,
        4.80797985e-01, 5.34340869e-01, 6.66952473e-02, 8.04141616e-03,
        1.01887489e-01, 4.60069740e-01, 5.60476165e-01, 1.78423621e-01,
        8.07273740e-01, 2.11812492e-02, 2.85201528e-02, 2.08197

합성곱 연산 구현의 문제점 - for문을 겹겹이 써야함
넘파이에 for문을 사용하면 성능이 떨어진다는 단점도 있음

im2col 함수를 사용한 구현
입력 데이터를 필터링하기 좋게 전해하는 함수
3차원 입력 데이터에 im2col을 적용하면 2차원 행렬도 바뀜
메모리를 더 많이 소비하는 단점이 있음
문제를 행렬 계산으로 만들면 선형 대수 라이브러리를 활용해 효율을 높일 수 있음

im2col 함수의 인터페이스
im2col(input_data, filter_h, filter_w, stride=1, pad=0)

In [5]:
import sys, os
import numpy as np
sys.path.append(os.pardir)
from common.util import im2col

In [8]:
x1 = np.random.rand(1, 3, 7, 7) # 데이터 수, 채널 수, 높이, 너비
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [11]:
x2 = np.random.rand(10, 3, 7, 7) # 데이터 수, 채널 수, 높이, 너비
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


합성곱 계층은 필터(가중치), 편향, 스트라이드, 패딩을 인수로 받아 초기화한다.
필터는 (FN, C, FH, FW)의 4차원 형상이다. FN = 필터의 개수, C = 채널, FH = 필터의 높이, FW = 필터의 너비

In [12]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad) # 1. 입력데이터를 im2col로 전개
        col_W =self.W.reshape(FN, -1).T # 2. 필터도 reshape을 사용해 2차원 배열로 전개
                                        # reshape에 -1을 지정하면 다차원 배열의 원소 수가 변환 후에도 똑같이 유지 되도록 적절히 묶어줌
        out = np.dot(col, col_W) + self.b # 3. 전개한 두 행렬의 곱을 구함

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # 4. 출력 데이터를 적절한 형상으로 바꿔줌
                                                                    # transpose 함수를 사용하여 다차원 배열의 축 순서를 바꿈
        return out

폴링 계층의 구현 세 단계
1. 입력 데이터를 전개한다.
2. 행별 최댓값을 구한다.
3. 적절한 모양으로 성형한다.

In [14]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool.h) / self.stride)
        out_w = int(1 + (W - self.pool.w) / self.stride)

        # (1) 전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # (2) 최댓값
        out = np.max(col, axis=1)

        # (3) 성형
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

CNN 계층 구현하기
다음과 같은 CNN을 구현한다.
Convolution(입력 데이터의 특징을 추출) -> ReLU -> Pooling(적절한 모양으로 성형) -> Affine(행렬곱 연산) -> ReLU -> Affine -> Softmax

In [15]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """단순한 합성곱 신경망
    
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]


In [18]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 10

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


train loss:2.2998736906960646
=== epoch:1, train acc:0.245, test acc:0.167 ===
train loss:2.2969574745443717
train loss:2.292817184859777
train loss:2.285985967193147
train loss:2.277800926408448
train loss:2.2586606879580704
train loss:2.2456149068433797
train loss:2.22573383630323
train loss:2.2237721108425945
train loss:2.1887619911291476
train loss:2.1607581239353046
train loss:2.125691619595951
train loss:2.048228123977685
train loss:1.9819903605538038
train loss:1.936836969611781
train loss:1.842447593116229
train loss:1.8113295100776778
train loss:1.6880248103143083
train loss:1.636505457809488
train loss:1.5544841339045161
train loss:1.468885208613945
train loss:1.3129450550650807
train loss:1.321690012605316
train loss:1.341452893286671
train loss:1.0537773765825644
train loss:1.050775823018068
train loss:0.9966729824070447
train loss:0.8260071139699728
train loss:0.8519184723667043
train loss:0.9679702475749475
train loss:0.8158377563714373
train loss:0.7817146870004609
train

KeyboardInterrupt: 